# Import Package

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage
import cv2
from skimage import io, measure, draw
from PIL import Image
import argparse
import json                              
from skimage import measure                        
from shapely.geometry import Polygon, MultiPolygon 
import os

# Crop Image 

In [ ]:
def crop(img, width, height):
    w_1 = int(width/2) -1500
    w_2 = int(width/2) + 1500
    h_1 = int(height/2) -1000
    h_2 = int(height/2) + 2000
    img2 = img.crop((w_1,h_1, w_2, h_2))
    
    return img2

# Mask

In [ ]:
def mask(img):
    image = np.array(img) 
    mask = cv2.circle(image, (1500, 0), 850, (0,0,0), -1) 
    return mask

# Image Processing

In [ ]:
def image_matting(img):
    processed = img.copy()

    processed = processed.convert('HSV')

    newImage = []
    for item in processed.getdata():
        if item[:3] < (45, 25, 30) and item[:3] > (15, 20, 30) :  # 下次從這看 2022/07/26
            newImage.append(item)
        else:
            newImage.append((0, 0, 0))

    processed.putdata(newImage)
    processed = processed.convert(mode="RGB")
    
    open_cv_image = np.array(processed) 
    
    # Convert RGB to BGR 
    open_cv_image = open_cv_image[:, :, ::-1].copy() 
    result = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
    
    (thresh, result) = cv2.threshold(result, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    return result 

# Crop and process and move to the final place 
### root(input_dir) is for folders that only root
### net(input_dir) is for folders that root including net

In [ ]:
def net(input_dir):
    for path, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.JPG'):
                # Output dictionary
                output_dir = os.path.join(path,'label')
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                output_path = os.path.join(output_dir, file)
                
                # Cropping dictionary
                crop_dir = os.path.join(path, 'crop')
                if not os.path.exists(crop_dir):
                    os.makedirs(crop_dir)
                crop_path = os.path.join(crop_dir, file)
                
                # Input image
                img_path = os.path.join(path, file)
                img = Image.open(img_path)
                w = img.width; h = img.height
                img_crop = crop(img,w,h)
                
                # Saving cropping image
                Image.Image.save(img_crop,crop_path)
                
                #Do image processing and save
                result = image_matting(img_crop)
                result = mask(result)
                cv2.imwrite(output_path,result)
                
    print('done')


In [ ]:
def root(input_dir):
    for path, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.JPG'):
                # Output dictionary
                output_dir = os.path.join(path,'label')
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                output_path = os.path.join(output_dir, file)
                
                # Cropping dictionary
                crop_dir = os.path.join(path, 'crop')
                if not os.path.exists(crop_dir):
                    os.makedirs(crop_dir)
                crop_path = os.path.join(crop_dir, file)
                
                # Input image
                img_path = os.path.join(path, file)
                img = Image.open(img_path)
                w = img.width; h = img.height
                img_crop = crop(img,w,h)
                
                # Saving cropping image
                Image.Image.save(img_crop,crop_path)
                
                #Do image processing and save
                result = image_matting(img_crop)
                cv2.imwrite(output_path, result)
    print('done')

In [ ]:
input_dirs = [
    "CK1-1"
]

In [ ]:
root(input_dirs[0])